In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np

In [2]:
outt = np.random.rand(3,3,4).astype("uint8")
#print(outt.shape)
w = torch.ones(4,4,3,3)
inp = torch.randn(1,4, 3, 3 )
conv1 = nn.Conv2d(4, 4, 3, 1,1)
conv1.weight = torch.nn.parameter.Parameter(w)
print (conv1.weight.shape)
print(conv1.bias)
#inp.transpose()
out = conv1(inp)
print (out.shape)
out = out.squeeze().transpose(0,2)

outt[0:3,0:3] = out.detach()
print(out)

torch.Size([4, 4, 3, 3])
Parameter containing:
tensor([-0.1533, -0.0342,  0.1042, -0.1627], requires_grad=True)
torch.Size([1, 4, 3, 3])
tensor([[[ 4.1657,  4.2849,  4.4233,  4.1563],
         [ 3.5251,  3.6443,  3.7827,  3.5157],
         [ 0.3740,  0.4931,  0.6316,  0.3646]],

        [[ 3.0414,  3.1606,  3.2990,  3.0320],
         [ 3.0528,  3.1719,  3.3103,  3.0434],
         [-0.0971,  0.0220,  0.1604, -0.1065]],

        [[ 5.0346,  5.1538,  5.2922,  5.0252],
         [ 5.8860,  6.0051,  6.1435,  5.8766],
         [ 2.6092,  2.7283,  2.8667,  2.5998]]], grad_fn=<TransposeBackward0>)


In [3]:
conv1 = nn.Conv2d(1, 1, 3, 1,1)
print (conv1.weight.shape)
print (conv1.bias.shape)

torch.Size([1, 1, 3, 3])
torch.Size([1])


In [4]:
inp = torch.arange(9).reshape(1,1, 3, 3 ).float()
w = torch.ones(1,1,3,3)
b = torch.zeros(1)
conv1 = nn.Conv2d(1, 1, 3, 1,1)
conv1.weight = torch.nn.parameter.Parameter(w)
conv1.bias = torch.nn.parameter.Parameter(b)
out = conv1(inp)
print(inp)
print (out)

tensor([[[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]]]])
tensor([[[[ 8., 15., 12.],
          [21., 36., 27.],
          [20., 33., 24.]]]], grad_fn=<ThnnConv2DBackward>)


In [5]:
out[out >15] = 15
print(out)

tensor([[[[ 8., 15., 12.],
          [15., 15., 15.],
          [15., 15., 15.]]]], grad_fn=<IndexPutBackward>)


In [6]:
print(conv1.weight)

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)


In [7]:
inp = torch.arange(49).reshape(1,1, 7, 7 ).float()
maxpool = nn.MaxPool2d(2, stride=2)
out = maxpool(inp)
print(inp)
print(out)



tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.],
          [ 7.,  8.,  9., 10., 11., 12., 13.],
          [14., 15., 16., 17., 18., 19., 20.],
          [21., 22., 23., 24., 25., 26., 27.],
          [28., 29., 30., 31., 32., 33., 34.],
          [35., 36., 37., 38., 39., 40., 41.],
          [42., 43., 44., 45., 46., 47., 48.]]]])
tensor([[[[ 8., 10., 12.],
          [22., 24., 26.],
          [36., 38., 40.]]]])


In [10]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.conv1 = nn.Conv2d(1,1,3)
        self.conv2 = nn.Conv2d(1,1,3)
        self.maxpool1 = nn.MaxPool2d(2, stride=2)
        self.maxpool2 = nn.MaxPool2d(2, stride=2, return_indices=True)
        self.hidden_layer = nn.Linear(144,121)
        self.dconv1 = nn.ConvTranspose2d(1, 1, 3, stride=1)
        self.dconv2 = nn.ConvTranspose2d(1, 1, 4, stride=2)
        self.dconv3 = nn.ConvTranspose2d(1, 1, 3, stride=1)
        self.unmaxpool1 = nn.MaxUnpool2d(2,stride=2)
        self.unmaxpool2 = nn.MaxUnpool2d(2,stride=2)
        
    def forward(self, features):
        code = self.conv1(features)
        print(code.shape)
        code = self.conv2(code)
        print(code.shape)
        code,indices = self.maxpool2(code)
        code = code.view(-1,144)
        print("hidden in shape = " + str(code.shape))
        code = self.hidden_layer(code)
        code = code.view(16,1,11,11)
        #code = self.unmaxpool1(code,indices,torch.Size([16, 1, 11, 11]))
        #print("unpool out shape = " + str(code.shape))
        code = self.dconv1(code)
        print(code.shape)
        code = self.dconv2(code)
        print(code.shape)
        return code


In [11]:
inp = torch.arange(16 * 28 * 28).reshape(16,1, 28, 28 ).float()
model = AE()
out = model(inp)

torch.Size([16, 1, 26, 26])
torch.Size([16, 1, 24, 24])
hidden in shape = torch.Size([16, 144])
torch.Size([16, 1, 13, 13])
torch.Size([16, 1, 28, 28])
